In [1]:
!pip install lime --user
!pip install tabulate --user
!pip install pdfkit --user

from sklearn import model_selection
from sklearn import ensemble
from sklearn import datasets

from lime import lime_tabular

import os
from multiprocessing import Process
import pandas as pd
import numpy as np
import sklearn
import lime
import pickle

PICKLES_PATH = "/afs/inesc-id.pt/home/aof/thesis/Parkinson-speech-detection/classification/pickles"
SUBSETS_PATH = "/afs/inesc-id.pt/home/aof/thesis/Parkinson-speech-detection/subsets_useful"
EXPLANATIONS_PATH = "/afs/inesc-id.pt/home/aof/public_html"

BOOTSTRAP = '<head><link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.1.3/css/bootstrap.min.css" integrity="sha384-MCw98/SFnGE8fJT3GXwEOngsV7Zt27NXFoaoApmYm81iuXoPkFOJwJ8ERdknLPMO" crossorigin="anonymous"></head>'
#=================================================================================
'''
iris = datasets.load_iris()

train, test, labels_train, labels_test = sklearn.model_selection.train_test_split(iris.data, iris.target, train_size=0.80)

rf = sklearn.ensemble.RandomForestClassifier(n_estimators=500)
rf.fit(train, labels_train)

explainer = lime.lime_tabular.LimeTabularExplainer(train, feature_names=iris.feature_names, class_names=iris.target_names, discretize_continuous=True)

for i in range(test.shape[0]):
    exp = explainer.explain_instance(test[i], rf.predict_proba, num_features=3, top_labels=2)
    print(vars(exp))
    exp.show_in_notebook(show_table=True, show_all=True)   
    #break
    print(vars(exp.domain_mapper))
    break
'''

EXPLANATIONS = {
    'f0': 'The number of open/close cycles of the glottis.',
    'hnr': 'Measures the frequency variation between cycles. Affected by lack of control on the vocal cords vibration.',
    'jitter': 'Measures the amplitude variation between cycles.',
    'shimmer': 'The ratio between periodic (associated with normal speed production) and non-periodic (associated with noise) speech components.',
    'mfcc': 'Features that approximate to our perception of the audio quality.',
    'plp': 'Features that approximate to our perception of the audio quality.',  
}


NORMAL_VALUES = {
    'f0': '105-160 (male) | 175-245 (Female)',
    'hnr': '< 1.04',
    'jitter': '< 3.81',
    'shimmer': '< 40',
    'mfcc': '',
    'plp': '',  
}

In [2]:
def write_html_to_file(html, path):
    f = open(path, "w")
    f.write(html)
    f.close()

In [3]:
def get_explanation_text(feature):
    return get_explanation_or_values(feature, EXPLANATIONS)

def get_normal_values(feature):
    return get_explanation_or_values(feature, NORMAL_VALUES)

def get_explanation_or_values(feature, obj):
    lower = feature.lower()
    
    for key in obj:
        if key in lower:
            return obj[key]
    
    print("No explanation or normal value found for feature {}".format(feature))
    return None

In [4]:
'''
    Model: MLPClassifier instance
    Patient_Lines: lines of CSV corresponding to the patient to be tested
    Train: Training fractionf of the CSV
    Feature_names: list of features (columns of csv)
    Output path: file path to write the output, None for writing on screen
'''
def explain_patient(experiment, patient, model, patient_lines, train, feature_names, params, output_path=None):
    
    num_features = 5
    
    #Build tabular
    explainer = lime.lime_tabular.LimeTabularExplainer(train, feature_names=feature_names, class_names=['HC', 'PD'], discretize_continuous=True)

    # Pass each line to LimeTabularExplainer
    explanations = []
    for i, row in patient_lines.iterrows():
        #Run only 1 in every 10 time frames (10 samples / second)
        if i % 10 == 0:
            explanations.append(explainer.explain_instance(row, model.predict_proba, num_features=num_features, top_labels=1))

    print("Got {} explanations".format(len(explanations)))
    # Sum all local_exp
    total_local_exp = {}
    for explanation in explanations:
        #local_exp = {1: [(12, 0.139704949653412), (44, 0.10278682378094679)]}
        local_exp = explanation.local_exp
        
        # will produce {<feature>: (count, sum_weight), ...
        for tup in local_exp[0 if 0 in local_exp else 1]:
            #tup = (12, 0.139704949653412)
            feature = feature_names[tup[0]]
            if feature in total_local_exp:
                total_local_exp[feature][0] += 1
                total_local_exp[feature][1] += tup[1]
            else:
                total_local_exp[feature] = [1, tup[1]]
    
    #Average local_exp
    # Key --> feature
    # value --> average value
    averaged_local_exp = {}

    for key in total_local_exp.keys():
        tup = total_local_exp[key]
        averaged_local_exp[key] = tup[1] / tup[0]

    # Normalize predict_proba
    average_predict_proba = [sum(items) for items in zip(*[explanation.predict_proba for explanation in explanations ])]
    average_predict_proba /= np.float64(sum(average_predict_proba))

    result = display_or_save_patient(averaged_local_exp, average_predict_proba, output_path, num_features, patient)
    f = open("/afs/inesc-id.pt/home/aof/thesis/Parkinson-speech-detection/explainability/logs/done_patients/{}/{}_{}.txt".format(experiment, generate_suffix(params), patient.split("_")[0]), "w")
    f.write("Done")
    f.close()
    return result

In [5]:
def generate_suffix(params):
    return "{}_{}_{}_{}".format(params['solver'], params['alpha'], params['max_iter'], params['activation'])

In [6]:
from IPython.display import HTML, display
import tabulate
import json
import traceback

def display_or_save_patient(averaged_local_exp, average_predict_proba, output_path, num_features, patient):
    prediction_proba = \
            '<table class="table" style="margin: 0 auto;"> \
                <thead> \
                    <tr> \
                        <td style="margin: 2px; text-align:center; font-weight: bold;">HC</td> \
                        <td style="margin: 2px; text-align:center; font-weight: bold;">PD</td> \
                    </tr> \
                </thead> \
                <tr> \
                    <td style="margin: 2px; text-align: center;">{}</td> \
                    <td style="margin: 2px; text-align: center;">{}</td> \
                </tr> \
            </table>' \
            .format("{:.2f}".format(average_predict_proba[0]), "{:.2f}".format(average_predict_proba[1]))
    
    averaged_local_exp = {k: v for k, v in sorted(averaged_local_exp.items(), key=lambda item: -abs(item[1]))}
    feature_weights = averaged_local_exp.items()
    feature_weight_table = '<table class="table" style="margin: 0 auto;"> \
                                <thead> \
                                    <tr> \
                                        <td style="margin: 2px; text-align:center; font-weight: bold;">Feature</td> \
                                        <td style="margin: 2px; text-align:center; font-weight: bold;">Relevance weight</td> \
                                        <td style="margin: 2px; text-align:center; font-weight: bold;">Healthy values</td> \
                                        <td style="margin: 2px; text-align:center; font-weight: bold;">Description</td> \
                                    </tr> \
                                </thead>'

    accounted = 0
    for feature_weight in feature_weights:
        s = '<tr> \
                <td style="margin: 2px; text-align:center; font-weight: bold;">{}</td> \
                <td style="margin: 2px; text-align:center;">{}</td> \
                <td style="margin: 2px; text-align:center;">{}</td> \
                <td style="margin: 2px; text-align:center;">{}</td> \
            </tr>' \
            .format(feature_weight[0], feature_weight[1], get_normal_values(feature_weight[0]), get_explanation_text(feature_weight[0]))
        feature_weight_table += s
        
        accounted += 1
        if accounted == num_features:
            break

    feature_weight_table += '</table>'
    
    #for feature_index in averaged_local_exp:
    #    feature_weight.append([feature_index, averaged_local_exp[feature_index]])
    
    
    table = '<table class="table" style="margin: 0 auto;"> \
                <thead> \
                    <tr> \
                        <td style="width: 45%; margin: 2px; text-align:center; font-weight: bold;">Prediction Probabilities</td> \
                        <td style="width: 45%; margin: 2px; text-align:center; font-weight: bold;">Feature weight</td> \
                    </tr> \
                </thead> \
                <tr> \
                    <td style="width: 45%; margin: 2px; text-align:center">{}</td> \
                    <td style="width: 45%; margin: 2px; text-align:center">{}</td> \
                </tr> \
            </table>' \
        .format(prediction_proba, feature_weight_table)

    if output_path == None:
        display(HTML(table))
    else:
        write_html_to_file(BOOTSTRAP + table, output_path)
        
    dic = {'name': patient.split("_")[0], 'hc': average_predict_proba[0], 'pd': average_predict_proba[1], 'features': [json.dumps(averaged_local_exp)]}
    try: 
        results = pd.DataFrame(dic, index=None)
    except: 
        message = traceback.format_exc()
        print(message)
    return results

In [7]:
def generate_mlp_params_list(dry_run=False):
    if (dry_run):
        return {
            "independent_200": [
                { # Got it
                    'dataset': 'gita',
                    'solver': 'lbfgs',
                    'activation': 'tanh',
                    'alpha': 0.0001,
                    'max_iter': 2000,
                },
            ]
        }
        
    return {
        "baseline": [
            { # Got it
                'dataset': 'gita',
                'solver': 'adam',
                'activation': 'tanh',
                'alpha': 0.0001,
                'max_iter': 5000,
            },
            { # Got it
                'dataset': 'gita',
                'solver': 'lbfgs',
                'activation': 'tanh',
                'alpha': 0.0001,
                'max_iter': 2000,
            },
            { # Got it
                'dataset': 'gita',
                'solver': 'adam',
                'activation': 'tanh',
                'alpha': 0.001,
                'max_iter': 2000,
            },
            { # Got it, on the hlt machine
                'dataset': 'gita',
                'solver': 'lbfgs',
                'activation': 'tanh',
                'alpha': 0.01,
                'max_iter': 5000,
            },
            { # Got it
                'dataset': 'gita',
                'solver': 'lbfgs',
                'activation': 'tanh',
                'alpha': 0.01,
                'max_iter': 2000,
            },
        ],
        'baseline_200': [
            { # Got it
                'dataset': 'gita',
                'solver': 'adam',
                'activation': 'tanh',
                'alpha': 0.001,
                'max_iter': 5000,
            },
            { # Got it
                'dataset': 'gita',
                'solver': 'lbfgs',
                'activation': 'tanh',
                'alpha': 0.001,
                'max_iter': 5000,
            },
            { # Got it
                'dataset': 'gita',
                'solver': 'adam',
                'activation': 'tanh',
                'alpha': 0.0001,
                'max_iter': 5000,
            },
            { # Got it
                'dataset': 'gita',
                'solver': 'adam',
                'activation': 'tanh',
                'alpha': 0.01,
                'max_iter': 5000,
            },
            { # Got it
                'dataset': 'gita',
                'solver': 'adam',
                'activation': 'tanh',
                'alpha': 0.001,
                'max_iter': 2000,
            },
        ],
        'semi': [
            { # Got it, on the hlt machine
                'dataset': 'mdvr_kcl_gita',
                'solver': 'adam',
                'activation': 'tanh',
                'alpha': 0.001,
                'max_iter': 2000,
            },
            { # Got it
                'dataset': 'fralusopark_gita',
                'solver': 'lbfgs',
                'activation': 'tanh',
                'alpha': 0.0001,
                'max_iter': 5000,
            },
            { # Got it
                'dataset': 'gita_fralusopark',
                'solver': 'adam',
                'activation': 'tanh',
                'alpha': 0.0001,
                'max_iter': 2000,
            },
            { # Got it, on the hlt machine
                'dataset': 'gita_fralusopark',
                'solver': 'adam',
                'activation': 'tanh',
                'alpha': 0.01,
                'max_iter': 5000,
            },
            { # Got it, on the hlt machine
                'dataset': 'gita_fralusopark',
                'solver': 'lbfgs',
                'activation': 'tanh',
                'alpha': 0.001,
                'max_iter': 2000,
            },
        ],
        'semi_200': [
            { # Got it, on the hlt machine
                'dataset': 'mdvr_kcl_gita',
                'solver': 'lbfgs',
                'activation': 'tanh',
                'alpha': 0.001,
                'max_iter': 5000,
            },
            { # Got it
                'dataset': 'fralusopark_gita',
                'solver': 'lbfgs',
                'activation': 'tanh',
                'alpha': 0.0001,
                'max_iter': 2000,
            },
            { # Cheated | Got it, on the hlt machine
                'dataset': 'mdvr_kcl_gita',
                'solver': 'adam',
                'activation': 'tanh',
                'alpha': 0.001,
                'max_iter': 2000,
            },
            { # Got it, on the hlt machine
                'dataset': 'mdvr_kcl_gita',
                'solver': 'adam',
                'activation': 'tanh',
                'alpha': 0.001,
                'max_iter': 5000,
            },
            { # Got it, on the hlt machine
                'dataset': 'mdvr_kcl_gita',
                'solver': 'lbfgs',
                'activation': 'tanh',
                'alpha': 0.0001,
                'max_iter': 5000,
            },
        ],
        'independent': [
            { # Got it, on the hlt machine
                'dataset': 'gita',
                'solver': 'lbfgs',
                'activation': 'tanh',
                'alpha': 0.001,
                'max_iter': 5000,
            },
            { # Got it, on the hlt machine
                'dataset': 'gita',
                'solver': 'lbfgs',
                'activation': 'tanh',
                'alpha': 0.01,
                'max_iter': 2000,
            },
            { # Got it, on the hlt machine
                'dataset': 'gita',
                'solver': 'lbfgs',
                'activation': 'tanh',
                'alpha': 0.001,
                'max_iter': 2000,
            },
            { # Cheated, train and test are from lbfgs, 0.01, 5000 | Got it
                'dataset': 'gita',
                'solver': 'lbfgs',
                'activation': 'tanh',
                'alpha': 0.0001,
                'max_iter': 2000,
            },
            { # Got it, on the hlt machine
                'dataset': 'gita',
                'solver': 'adam',
                'activation': 'tanh',
                'alpha': 0.0001,
                'max_iter': 5000,
            },
        ],
        'independent_200': [
            { # Got it, on the hlt machine
                'dataset': 'gita',
                'solver': 'lbfgs',
                'activation': 'tanh',
                'alpha': 0.01,
                'max_iter': 5000,
            },
            { # Got it
                'dataset': 'gita',
                'solver': 'lbfgs',
                'activation': 'tanh',
                'alpha': 0.0001,
                'max_iter': 2000,
            },
            { # Got it
                'dataset': 'gita',
                'solver': 'lbfgs',
                'activation': 'tanh',
                'alpha': 0.0001,
                'max_iter': 5000,
            },
            { # Got it, on the hlt machine
                'dataset': 'gita',
                'solver': 'lbfgs',
                'activation': 'tanh',
                'alpha': 0.001,
                'max_iter': 2000,
            },
            { # Got it
                'dataset': 'fralusopark',
                'solver': 'lbfgs',
                'activation': 'tanh',
                'alpha': 0.0001,
                'max_iter': 5000,
            },
        ]
    }

In [9]:
import traceback

def explain(train_path, test_path, experiment, param_set, logs_file, dry_run=False):
    pickle_file = '{}/{}/{}/{}.pkl'.format(PICKLES_PATH, experiment, param_set['dataset'], generate_suffix(param_set))
    if os.path.isfile(pickle_file):
        with open(pickle_file, 'rb') as filename:
            model = pickle.load(filename)
    else:
        logs_file.write("WARNING: No pickle found @ {}. Skipping...\n".format(pickle_file))
        return None

    if not os.path.isfile(train_path):
        logs_file.write("WARNING: No train file found @ {}. Skipping...\n".format(train_path))
        return train_path
    if not os.path.isfile(test_path):
        logs_file.write("WARNING: No test file found @ {}. Skipping...\n".format(test_path))
        return test_path

    train = None
    test = None
    try:
        train = pd.read_csv(train_path, sep=";")
    except Exception as e:
        logs_file.write('Test file reading error: ' + str(e))
    try:
        test = pd.read_csv(test_path , sep=";")
    except Exception as e:
        logs_file.write('Test file reading error: ' + str(e))
                

    train = train[train.columns.difference(['frameTime', 'label', 'name'])]

    patients = list(set(test['name']))
    feature_names = [feature for feature in list(test.columns) if feature not in ['label', 'name', 'frameTime']]
    
    output_dic = {'name': [], 'hc': [], 'pd': [], 'features': []}
    output_csv = pd.DataFrame(output_dic)
    
    try:
        for patient in patients:
            patient_lines = test[test["name"] == patient]
            patient_lines = patient_lines[patient_lines.columns.difference(['frameTime', 'label', 'name'])]

            output_path_html = "{}/{}/{}/{}_{}.html".format(EXPLANATIONS_PATH, experiment, param_set['dataset'], patient.split('_')[0], generate_suffix(param_set))

            if os.path.isfile(output_path_html):
                print("Explanation exists, ignoring...")
                continue

            logs_file.write("Explaining patient {} to {}\n".format(patient.split('_')[0], output_path_html))
            if not dry_run:
                print(patient)
                try:
                    patient_explanation = explain_patient(experiment, patient, model, patient_lines, train.to_numpy(), feature_names, param_set, output_path=output_path_html)
                    output_csv = output_csv.append(patient_explanation, ignore_index=True)
                except:
                    logs_file.write('Error explaining patient {} for experiment {}\n'.format(patient, experiment))
    except Exception:
        output_csv.to_csv('/afs/inesc-id.pt/home/aof/thesis/Parkinson-speech-detection/explainability/csv/{}/{}'.format(experiment, generate_suffix(param_set)), index=False)
        message = traceback.format_exc()
        logs_file.write(message + '\n')
        print(message)
    print(output_csv)
    output_csv.to_csv('/afs/inesc-id.pt/home/aof/thesis/Parkinson-speech-detection/explainability/csv/{}/{}.csv'.format(experiment, generate_suffix(param_set)), index=False)
    f = open("/afs/inesc-id.pt/home/aof/thesis/Parkinson-speech-detection/explainability/logs/done_experiments/{}.txt".format(experiment), "w")
    f.write("Done")
    f.close()
    return None

In [10]:
def get_train_test_path(experiment, param_set):
    train = "{}/{}/{}/{}_train.csv".format(SUBSETS_PATH, experiment, param_set['dataset'], generate_suffix(param_set))
    test =  "{}/{}/{}/{}_test.csv".format (SUBSETS_PATH, experiment, param_set['dataset'], generate_suffix(param_set))  
    
    return train, test

In [11]:
def get_datasets_names():
    return {
        'baseline': ['fralusopark', 'gita', 'mdvr_kcl'],
        'baseline_200': ['fralusopark', 'gita', 'mdvr_kcl'],
        'independent': ['fralusopark', 'gita', 'mdvr_kcl'],
        'independent_200': ['fralusopark', 'gita', 'mdvr_kcl'],
        'semi': ['fralusopark_gita', 'fralusopark_mdvr_kcl', 'gita_fralusopark', 'gita_mdvr_kcl', 'mdvr_kcl_fralusopark', 'mdvr_kcl_gita'],
        'semi_200': ['fralusopark_gita', 'fralusopark_mdvr_kcl', 'gita_fralusopark', 'gita_mdvr_kcl', 'mdvr_kcl_fralusopark', 'mdvr_kcl_gita']
    }

In [12]:
def run_experiment_patients(experiment, mlp_params):
    missing_models = []
    f = open("/afs/inesc-id.pt/home/aof/thesis/Parkinson-speech-detection/explainability/logs/logs_{}.txt".format(experiment), "w")
    for param_set in mlp_params:
        train_path, test_path = get_train_test_path(experiment, param_set)
        result = explain(train_path, test_path, experiment, param_set, f)
        
        if result:
            missing_models.append(result)
    f.close()
      
    if missing_models != []:
        f = open('/afs/inesc-id.pt/home/aof/thesis/Parkinson-speech-detection/explainability/logs/missing_modules/{}.txt'.format(experiment), "w")
        f.write('\n'.join(missing_models))
        f.close()


In [13]:
def main():
    mlp_params = generate_mlp_params_list(dry_run=True)

    experiment_processes = []
    for experiment in mlp_params.keys():
        experiment_processes.append(Process(target=run_experiment_patients, args=(experiment, mlp_params[experiment],)))

    for experiment_process in experiment_processes:
        experiment_process.start()

    for experiment_process in experiment_processes:
        experiment_process.join()


In [14]:
if __name__ == '__main__':
    main()

/usr/lib64/python3.6/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/lib64/python3.6/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator MLPClassifier from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


AVPEPUDEAC0018_readtext
AVPEPUDEA0010_readtext
AVPEPUDEAC0057_readtext
AVPEPUDEAC0054_readtext
AVPEPUDEA0050_readtext
AVPEPUDEAC0041_readtext
AVPEPUDEA0030_readtext
AVPEPUDEAC0051_readtext
AVPEPUDEA0059_readtext
AVPEPUDEAC0014_readtext
AVPEPUDEA0008_readtext
AVPEPUDEA0002_readtext
AVPEPUDEAC0017_readtext
AVPEPUDEA0013_readtext
AVPEPUDEAC0052_readtext
AVPEPUDEAC0047_readtext
AVPEPUDEA0039_readtext
AVPEPUDEAC0023_readtext
AVPEPUDEA0020_readtext
AVPEPUDEAC0006_readtext
AVPEPUDEAC0044_readtext
AVPEPUDEAC0020_readtext
AVPEPUDEAC0010_readtext
AVPEPUDEAC0005_readtext
AVPEPUDEA0015_readtext
AVPEPUDEA0024_readtext
AVPEPUDEA0057_readtext
AVPEPUDEAC0026_readtext
AVPEPUDEA0022_readtext
AVPEPUDEAC0024_readtext
AVPEPUDEAC0003_readtext
AVPEPUDEAC0022_readtext
AVPEPUDEA0016_readtext
AVPEPUDEA0035_readtext
AVPEPUDEA0043_readtext
AVPEPUDEA0045_readtext
AVPEPUDEA0009_readtext
AVPEPUDEAC0011_readtext
AVPEPUDEAC0048_readtext
AVPEPUDEAC0039_readtext
AVPEPUDEA0026_readtext
AVPEPUDEAC0016_readtext
AVPEPUDEAC0